# DAT210x - Programming with Python for DS

## Module6- Lab3

This code is intentionally missing! Read the directions on the course lab page!

In [179]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [180]:
# Read data. 
# Separate 'status' feature to 'y' Series
X = pd.read_csv('./Datasets/parkinsons.data')
y = X['status']
X = X.drop(columns=['status', 'name'])
X.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),...,MDVP:APQ,Shimmer:DDA,NHR,HNR,RPDE,DFA,spread1,spread2,D2,PPE
0,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,0.426,...,0.02971,0.06545,0.02211,21.033,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,0.626,...,0.04368,0.09403,0.01929,19.085,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,0.482,...,0.03590,0.08270,0.01309,20.651,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,0.517,...,0.03772,0.08771,0.01353,20.644,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,0.584,...,0.04465,0.10470,0.01767,19.649,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [181]:
X.dtypes

MDVP:Fo(Hz)         float64
MDVP:Fhi(Hz)        float64
MDVP:Flo(Hz)        float64
MDVP:Jitter(%)      float64
MDVP:Jitter(Abs)    float64
MDVP:RAP            float64
MDVP:PPQ            float64
Jitter:DDP          float64
MDVP:Shimmer        float64
MDVP:Shimmer(dB)    float64
Shimmer:APQ3        float64
Shimmer:APQ5        float64
MDVP:APQ            float64
Shimmer:DDA         float64
NHR                 float64
HNR                 float64
RPDE                float64
DFA                 float64
spread1             float64
spread2             float64
D2                  float64
PPE                 float64
dtype: object

In [182]:
y.head()

0    1
1    1
2    1
3    1
4    1
Name: status, dtype: int64

In [183]:
from sklearn.model_selection import train_test_split
# Test data size is 30% 
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=7)

# === Test preprocessing 
# Experiment with Normalizer(), MaxAbsScaler(), MinMaxScaler(), KernelCenterer(), and StandardScaler().\\
from sklearn import preprocessing 

#prep = preprocessing.Normalizer()
#prep = preprocessing.MaxAbsScaler()
#prep = preprocessing.MinMaxScaler()
#prep = preprocessing.KernelCenterer()
prep = preprocessing.StandardScaler()

prep.fit( X_train )
X_train = prep.transform( X_train )
X_test = prep.transform( X_test ) 

# === Test PCA 
"""
# Experiment n_compoenents 4 ~ 14 
n_components = 14

from sklearn.decomposition import PCA 
pca = PCA(n_components=n_components) 
pca.fit( X_train ) 
X_train = pca.transform( X_train ) 
X_test  = pca.transform( X_test  )
"""

#=== Test Isomap 
from sklearn.manifold import Isomap

n_componetns = 5
iso = Isomap(n_components=n_components)
iso.fit( X_train ) 
X_train = iso.transform( X_train ) 
X_test  = iso.transform( X_test  )



In [184]:
X_train.shape


(136, 14)

In [185]:
from sklearn.svm import SVC 

model = SVC() # C=1, kernel=linear, gamma='auto'
model.fit( X_train, y_train ) 
score = model.score( X_test, y_test )
print('Score = ', score)

Score =  0.9322033898305084


That accuracy was just too low to be useful. We need to get it up. One way you could go about doing that would be to manually try a bunch of combinations of C, and gamma values for your rbf kernel. But that could literally take forever. Also, you might unknowingly skip a pair of values that would have resulted in a very good accuracy.  

Instead, lets get the computer to do what computers do best. Program a naive, best-parameter search by creating nested for-loops. 

The outer for-loop should iterate a variable C from 0.05 to 2, using 0.05 unit increments. The inner for-loop should increment a variable gamma from 0.001 to 0.1, using 0.001 unit increments. 

As you know, Python ranges won't allow for float intervals, so you'll have to do some research on NumPy ARanges, if you don't already know how to use them.
Since the goal is to find the parameters that result in the model having the best accuracy score, you'll need a best_score = 0 variable that you initialize outside of the for-loops. Inside the inner for-loop, create an SVC model and pass in the C and gamma parameters its class constructor. Train and score the model appropriately. If the current best_score is less than the model's score, update the best_score being sure to print it out, along with the C and gamma values that resulted in it.

In [186]:
best_score = 0

for C in np.arange(0.05,2.05,0.05):    
    for gamma in np.arange(0.001, 0.1001, 0.001):
        model = SVC(kernel='rbf', C=C, gamma=gamma)
        model.fit( X_train, y_train )
        score = model.score( X_test, y_test )
        if score > best_score: 
            best_score = score
            best_C, best_gamma = C, gamma 
            
print('Score:C:gamma ', best_score, best_C, best_gamma)


Score:C:gamma  0.9491525423728814 1.3 0.097


Without PCA   
- StandardScaler:  0.9322033898305084

StarndardSaler & PCA 
- n_componnets=4  / 0.8983050847457628
- n_componnets=5  / 0.9152542372881356
- n_components=6  / 0.9152542372881356
- n_components=7  / 0.9322033898305084
- n_components=8  / 0.9322033898305084
- n_components=9  / 0.9322033898305084
- n_components=10 / 0.9322033898305084
- n_components=11 / 0.9322033898305084 
- n_components=12 / 0.9322033898305084
- n_components=13 / 0.9322033898305084 
- n_components=14 / 0.9322033898305084  

StandardScaler & Isomap  
- n_components=2 / 0.9491525423728814 
- n_components=3 / 0.9491525423728814  
- n_components=4 / 0.9491525423728814   
- n_components=5 / 0.9491525423728814    

